# Visualizing RST and layout structures in GeM corpora

This Jupyter notebook is intended for visualizing discourse structures in multimodal documents, as described using <a href="http://www.sfu.ca/rst/">Rhetorical Structure Theory</a> and annotated using the schema provided by the <a href="http://www.fb10.uni-bremen.de/anglistik/langpro/projects/gem/newframe.html">Genre and Multimodality</a> model.

For visualizing the RST and layout structures, you need valid XML files for base, RST and layout layers. The content is retrieved from the base layer, before drawing out the rhetorical relations between the content and organizing it into the hierarchy defined in the layout structure.

## 1. Import the necessary packages.

In [1]:
# For parsing XML
from lxml import etree as et

# For formatting the content
import textwrap
import codecs

# For testing
from IPython.display import Image

Check GraphViz installation.

In [2]:
graphviz_installed = !which dot
if graphviz_installed == '':
    print "Graphviz/DOT not found. Exiting ..."
    quit()
else:
    print "Graphviz/DOT found ..."

Graphviz/DOT found ...


## 2. Parse the GeM XML files.

Define the base and RST files below.

In [3]:
basefile = 'test_xml/2002-she-base-1.xml' # Base layer
rstfile = 'test_xml/2002-she-rst-1.xml' # RST layer
layoutfile = 'test_xml/2002-she-layout-1.xml' # Layout layer

Define a function to parse the XML files.

In [4]:
def parse_xml(basefile, rstfile, layoutfile):
    
    ##### PARSE BASE UNITS #####
    
    base_units = {} # Set up a dictionary for the base units
    baseroot = et.parse(basefile).getroot() # Parse the XML file and get XML root
    for unit in baseroot.findall('.//unit'):
        unit_id = unit.attrib['id']
        if 'alt' in unit.attrib:
            unit_content = unit.attrib['alt'], ', fontsize="12.0", shape="box"];\n'
        else:
            unit_content = textwrap.fill(''.join(unit.itertext()), 35).replace("\"", "'"), ', fontsize="12.0", shape="box", style="filled", color="white"];\n'
        base_units[unit_id] = unit_content # Populate dictionary with key (id) and value (content) pairs
            
    rst_segments = {} # Set up a dictionary for the RST units
    rst_base_mapping = {} # Set up a dictionary for mapping the RST units to the base units
    rstroot = et.parse(rstfile).getroot()
    
    ##### PARSE RST UNITS #####
    
    for rst_segment in rstroot.findall('.//segment'):
        rst_segment_id, rst_segment_xref = rst_segment.attrib['id'], rst_segment.attrib['xref']
        rst_segment_content = base_units[rst_segment_xref]
        rst_segments[rst_segment_id] = rst_segment_content
        rst_base_mapping[rst_segment_xref] = rst_segment_id
        
    for mini_segment in rstroot.findall('.//mini-segment'):
        mini_segment_id, mini_segment_xref = mini_segment.attrib['id'], mini_segment.attrib['xref']
        mini_segment_content = base_units[mini_segment_xref]
        rst_segments[mini_segment_id] = mini_segment_content
        rst_base_mapping[mini_segment_xref] = mini_segment_id
    
    rst_segment_span_mapping = {}
    rst_relations = {}

    for rst_structure in rstroot.findall('rst-structure'):
        for span in rst_structure.getchildren():
            rst_relations[span.attrib['id']] = span.attrib['relation'].upper(), ', fontcolor="crimson", fontsize="10.0", shape="plaintext"];\n'
            if span.tag == 'span':
                rst_segment_span_mapping[span.attrib['id']] = span.attrib['nucleus'].split(), span.attrib['satellites'].split()
            if span.tag == 'multi-span':
                rst_segment_span_mapping[span.attrib['id']] = span.attrib['nuclei'].split()
            if span.tag == 'mini-span':
                rst_segment_span_mapping[span.attrib['id']] = span.attrib['attribuend'].split(), span.attrib['attribute'].split()
    
    ##### PARSE LAYOUT UNITS #####

    layout_units = {} # Set up a dictionary for the layout units
    layoutroot = et.parse(layoutfile).getroot()

    for layout_unit in layoutroot.xpath('.//segmentation/layout-unit'): # Use XPath to skip nested layout units
        layout_unit_id = layout_unit.attrib['id']
        layout_unit_xref = layout_unit.attrib['xref'].split()
        layout_units[layout_unit_id] = layout_unit_xref

    ##### Parse the layout leafs ##### 
    
#    for layout_leaf in layoutroot.iter('layout-leaf'):
#        xref, layout_chunk, base_units = layout_leaf.attrib['xref'], layout_leaf.getparent().attrib['id'], layout_units[layout_leaf.attrib['xref']]
#        layout_units[xref] = base_units, layout_chunk
    
    return rst_segments, rst_relations, rst_base_mapping, rst_segment_span_mapping, rstroot, layout_units, layoutroot

In [5]:
rst_segments, rst_relations, rst_base_mapping, rst_segment_span_mapping, rstroot, layout_units, layoutroot = parse_xml(basefile, rstfile, layoutfile)

## 3. Draw the RST graph.

Open a file for the graph; use UTF-8 encoding. Set up the beginning and the end of the graph.

In [6]:
graph = codecs.open('rst_layout_graph.gv', 'w', 'utf-8')

begin_graph = 'digraph "rstgraph" { graph [rankdir="BT"];\n'
terminate_graph = '}\n'

Begin writing the graph.

In [34]:
##### NEW TEST #####

graph.write(begin_graph)

for number, subgraph in enumerate(layoutroot.xpath('.//*/../layout-chunk[@id]')):
    subgraph_id = subgraph.attrib['id']
    print '\tsubgraph ', 'cluster', str(number), ' {\n', 'label = "', subgraph_id, '";\n' # For testing
    write_chunk = '\tsubgraph ', 'cluster', str(number), ' {\n', 'label = "', subgraph_id, '";\n'
    graph.write("".join(write_chunk))

    # XPath expression for finding the identified chunks in the layout structure and their layout-leaf children   
    chunk_xpath = './/layout-chunk[@id="' + subgraph_id + '"]/layout-leaf[@xref]'
    # Loop over the layout leafs under the chunk ...
    for leafs in layoutroot.xpath('%s' % chunk_xpath):
        # ... and fetch the matching RST segments.
        for xref in layout_units[leafs.attrib['xref']]:
            # Exclude layout units not participating in RST structure
            if xref in rst_base_mapping:
                rst_segment = rst_base_mapping[xref]
                # Add edges for nucleus
                span_xpath = './/*[@nucleus="' + rst_segment + '"]/@id'
                for rst_span in rstroot.xpath('%s' % span_xpath):
                    nucleus_xpath = './/*[@id="' + rst_span + '"]/@nucleus'
                    for nucleus in rstroot.xpath('%s' % nucleus_xpath):
                        print '"', nucleus, '" -> ', '"', rst_span, '"\t [dir="back"];\n'

	subgraph  cluster 0  {
label = " page-10 ";

	subgraph  cluster 1  {
label = " page-11 ";

	subgraph  cluster 2  {
label = " chapter-title ";

	subgraph  cluster 3  {
label = " section-title-and-description ";

" s-10.04 " ->  " span-10.02 "	 [dir="back"];

" s-10.05 " ->  " span-10.03 "	 [dir="back"];

	subgraph  cluster 4  {
label = " page-11-entry-1 ";

" s-10.08 " ->  " span-10.06 "	 [dir="back"];

" s-10.11 " ->  " span-10.08 "	 [dir="back"];



In [41]:
##### EXAMINE THE LAYOUT STRUCTURE ##### 

# Map the RST segments to the layout chunks
rst_layout_chunk_mapping = {}
for chunks in layoutroot.findall('.//layout-chunk'): # Find all layout chunks
    chunk_id = chunks.attrib['id'] # Fetch unique chunk identifier    
    for leaf in layout_units: # Loop through layout units
        base_unit_id, chunk = layout_units.get(leaf) # Unpack layout units tuple
        if chunk == chunk_id:
            for base_unit in base_unit_id:
                if base_unit in rst_base_mapping.keys():
                    rst_layout_chunk_mapping[rst_base_mapping[base_unit]] = chunk

# Print the DOT graph preamble
# print begin_graph
graph.write(begin_graph)
    
# Add all RST spans
for span in sorted(rst_relations):
    relation, style = rst_relations.get(span)
    # print '"', span, '"\t[label="', relation, '"', style # For testing
    print_span = '"', span, '"\t[label="', relation, '"', style
    graph.write("".join(print_span))

# Add all RST segments
for node in sorted(rst_segments):
    content, style = rst_segments.get(node)
    # print '"', node, '"\t[label="', content, '"', style # For testing
    print_node = '"', node, '"\t[label="', content, '"', style
    graph.write("".join(print_node))
    
# Start writing the subgraphs
for number, chunk in enumerate(layoutroot.findall('.//layout-chunk')):

    if chunk.attrib['id'] in rst_layout_chunk_mapping.values():
        print '\tsubgraph ', 'cluster', str(number), ' {\n', 'label = "', chunk.attrib['id'], '";\n' # For testing
        print_chunk = '\tsubgraph ', 'cluster', str(number), ' {\n', 'label = "', chunk.attrib['id'], '";\n'
        graph.write("".join(print_chunk))

        for rst_segment, layout_chunk in rst_layout_chunk_mapping.items(): # Split the dictionary
            if chunk.attrib['id'] == layout_chunk: # Match chunks
                for span in rstroot.findall('.//rst-structure/*'): # Add edges for nuclei
                    if span.tag == 'span' and rst_segment in span.attrib['nucleus']:
                        nucleus = span.attrib['nucleus'].split()
                        for n in nucleus:
                            if rst_segment == n:
                                print '"', n, '" -> ', '"', span.attrib['id'], '"\t [dir="back"];\n' # For testing
                                print_nucleus = '"', n, '" -> ', '"', span.attrib['id'], '"\t [dir="back"];\n'
                                graph.write("".join(print_nucleus))
                    if span.tag == 'span' and rst_segment in span.attrib['satellites']:
                        satellites = span.attrib['satellites'].split()
                        for s in satellites:
                            if rst_segment == s:
                                print '"', s, '" -> ', '"', span.attrib['id'], '"\t [dir="none"];\n' # For testing
                                print_satellites = '"', s, '" -> ', '"', span.attrib['id'], '"\t [dir="none"];\n'
                                graph.write("".join(print_satellites))
        
        graph.write(terminate_graph)

for span in rstroot.findall('.//rst-structure/*'):
    if span.tag == 'multi-span':
        multispan, nuclei, relation = span.attrib['id'], span.attrib['nuclei'].split(), span.attrib['relation']
        if span.attrib['id'] == multispan:
            for n in nuclei:
                print_nuclei = '"', n, '" -> ', '"', multispan, '"\t [dir="back"];\n'
                graph.write("".join(print_nuclei))
                
    for title in span.iter('title'):
        target_id = span.attrib['id']
        title_xref = title.attrib['xref']
        if span.attrib['id'] == target_id:
            print_title = '"', title_xref, '" -> ', '"', target_id, '"\t [dir="none", style="dashed"];\n'
            graph.write("".join(print_title))
        
graph.write(terminate_graph)

ValueError: need more than 1 value to unpack

Close the graph file.

In [ ]:
graph.close()

Use DOT draw the graph.

In [ ]:
!dot -Tpng rst_layout_graph.gv > rst_layout_graph.png

Show the resulting image in the IPython notebook.

In [ ]:
Image('rst_layout_graph.png')